In [17]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import statsmodels.tsa.vector_ar.vecm as vm
from sklearn.linear_model import LinearRegression

df_bito = pd.read_csv("/home/nelson/PycharmProjects/my_notes/Dummy_data_for_examples/currency_etf/bito.csv", index_col=0)
df_bito.columns = df_bito.columns + "_bito"
df_btf = pd.read_csv("/home/nelson/PycharmProjects/my_notes/Dummy_data_for_examples/currency_etf/btf.csv", index_col=0)
df_btf.columns = df_btf.columns + "_btf"

df_data = df_btf.merge(df_bito, how="inner", left_on="date_btf", right_on="date_bito")
df_data

,date_btf,open_btf,high_btf,low_btf,close_btf,volume_btf,date_bito,open_bito,high_bito,low_bito,close_bito,volume_bito
0,2022-01-03T15:00:00.000Z,180.470,181.425,180.300,180.960,176244.0,2022-01-03T15:00:00.000Z,180.470,181.425,180.300,180.960,176244.0
1,2022-01-03T16:00:00.000Z,180.965,181.755,180.400,181.465,109457.0,2022-01-03T16:00:00.000Z,180.965,181.755,180.400,181.465,109457.0
2,2022-01-03T17:00:00.000Z,181.470,182.160,181.070,181.680,107664.0,2022-01-03T17:00:00.000Z,181.470,182.160,181.070,181.680,107664.0
3,2022-01-03T18:00:00.000Z,181.725,182.870,181.660,182.360,256132.0,2022-01-03T18:00:00.000Z,181.725,182.870,181.660,182.360,256132.0
4,2022-01-03T19:00:00.000Z,182.370,182.430,181.670,181.875,97949.0,2022-01-03T19:00:00.000Z,182.370,182.430,181.670,181.875,97949.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1801,2022-12-28T17:00:00.000Z,126.620,127.730,126.410,127.625,224599.0,2022-12-28T17:00:00.000Z,126.620,127.730,126.410,127.625,224599.0
1802,2022-12-28T18:00:00.000Z,127.655,128.385,126.845,127.080,149417.0,2022-12-28T18:00:00.000Z,127.655,128.385,126.845,127.080,149417.0
1803,2022-12-28T19:00:00.000Z,127.090,127.500,126.810,126.830,112721.0,2022-12-28T19:00:00.000Z,127.090,127.500,126.810,126.830,112721.0
1804,2022-12-28T20:00:00.000Z,126.835,126.840,125.880,126.070,374646.0,2022-12-28T20:00:00.000Z,126.835,126.840,125.880,126.070,374646.0


Item of note. The code below uses R style matrix formatting code. To get the correct coefficient ratio use this format.

Or use scikit learns linear regression model.
Otherwise, you can get the correct format by using a vector of 1's

In [28]:
print("Statsmodel.Formula.Api Method")
model1 = smf.ols(formula='open_btf ~ open_bito', data=df_data).fit()
print(model1.params)
print(model1.summary())

btf_open = df_data["open_btf"].tolist()  # y term
bito_open = df_data["open_bito"].tolist()  # x term

bito_open = sm.add_constant(bito_open)

print("Statsmodel.Api Method")
results = sm.OLS(btf_open, bito_open).fit()
bito_open = bito_open[1]
print(results.params)
print(results.summary())

print("Sci-Kit Learn Method")
model3 = LinearRegression()
# get data into correct format
btf_open = df_data["open_btf"].values # y term
bito_open = df_data["open_bito"].values  # x term
btf_open = btf_open.reshape(df_data.shape[0], 1)
bito_open = bito_open.reshape(df_data.shape[0], 1)

# or
X = df_data[["open_btf"]]
y = df_data[["open_bito"]]

model3.fit(bito_open, btf_open)  # is (X, Y) not (y, x) as above
print(model3.coef_)
print(model3.intercept_)

Statsmodel.Formula.Api Method
Intercept   -1.287859e-14
open_bito    1.000000e+00
dtype: float64
                            OLS Regression Results                            
Dep. Variable:               open_btf   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.005e+31
Date:                Tue, 03 Jan 2023   Prob (F-statistic):               0.00
Time:                        19:29:42   Log-Likelihood:                 50498.
No. Observations:                1806   AIC:                        -1.010e+05
Df Residuals:                    1804   BIC:                        -1.010e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------